In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from spotipy.oauth2 import SpotifyOAuth


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='d0e97913f362461786f58ba083820d8e',
    #client_secret='' insert secret from spotify developer dashboard
    redirect_uri='http://localhost:8890/callback',
    scope='playlist-modify-private playlist-modify-public playlist-read-private playlist-read-collaborative'
))

def get_all_tracks_from_user(user_id):
    print(f"Fetching playlists owned by user: {user_id}")
    track_ids = set()

    playlists = sp.user_playlists(user_id)
    while playlists:
        for playlist in playlists['items']:

            if playlist['owner']['id'] != user_id:
                continue

            playlist_id = playlist['id']
            print(f"  -> Scanning playlist: {playlist['name']}")

            results = sp.playlist_tracks(playlist_id)
            while results:
                for item in results['items']:
                    track = item['track']
                    if track:
                        track_ids.add(track['id'])
                results = sp.next(results) if results['next'] else None

        playlists = sp.next(playlists) if playlists['next'] else None

    return track_ids


def get_track_infos(track_ids):
    track_infos = []
    track_ids = list(track_ids)

    for i in range(0, len(track_ids), 50):
        batch = track_ids[i:i+50]
        try:
            results = sp.tracks(batch)
            for track in results['tracks']:
                if track:  # skip None
                    name = track['name']
                    artists = ', '.join([artist['name'] for artist in track['artists']])
                    track_infos.append(f"{name} by {artists}")
        except Exception as e:
            print(f"Error fetching batch {i//50 + 1}: {e}")
    
    return track_infos


def find_common_tracks(user1, user2):
    user1_tracks = get_all_tracks_from_user(user1)
    user2_tracks = get_all_tracks_from_user(user2)

    common = user1_tracks & user2_tracks
    print(f"\nFound {len(common)} shared tracks between {user1} and {user2}:\n")
    
    track_info_list = get_track_infos(common)
    for info in track_info_list:
        print(info)
    
    return common  

if __name__ == "__main__":
    user1 = input("Enter first Spotify username: ")
    user2 = input("Enter second Spotify username: ")
    common_track_ids = find_common_tracks(user1, user2) 

    # Create playlist
    create_playlist_with_tracks(sp.current_user()['id'], common_track_ids)


def create_playlist_with_tracks(user_id, track_ids, playlist_name="Shared Songs"):
    print(f"\nCreating playlist '{playlist_name}' for user {user_id}...")

    playlist = sp.user_playlist_create(
        user=user_id,
        name=playlist_name,
        public=False,
        description="Songs shared between users"
    )

    track_list = list(track_ids)
    for i in range(0, len(track_list), 100):
        sp.playlist_add_items(playlist['id'], track_list[i:i+100])

    print(f"Playlist '{playlist_name}' created with {len(track_list)} tracks.")
